In [20]:
import numpy as np
import pandas as pd

## Hyper Parameter

In [ ]:
POP_SIZE = 30

## Data Preparation

In [21]:
with open ("Dataset\Hurink_Data\\vdata\\abz5.fjs","r") as file:
    firstLine = file.readline()
    firstLineValues = [int(i) for i in firstLine.split()[0:2]]
    NUM_JOBS = firstLineValues[0] 
    NUM_MACHINES = firstLineValues[1]
    print(f"numJobs = {NUM_JOBS}, numMachines = {NUM_MACHINES}")
    
    data = []
    job_index = 0   
    for txts in file.readlines():
        job = []
        operations = txts.split()[1:]
        index = 0
        while index < len(operations):
            operation = []
            numberOfOperation = int(operations[index])
            index += 1
            endOperationIndex = numberOfOperation*2 + index
            while index < endOperationIndex:
                operation.append({"macineNumber": operations[index], "processTime": operations[index+1]})
                index += 2
            job.append(operation)
        data.append(job)
    
# data = [job1, job2, .... ]
# job1 = , [op1, op2, op3, ....]
# op1 = [{macineNumber: , processTime: },{machineNumber: , processTime: }...]
print(f"number of first job's operation = {len(data[0])}")
print(f"number of machine that first job's operation can use = {len(data[0][0])}")
print(data[0][0])

numJobs = 10, numMachines = 10
number of first job's operation = 10
number of machine that first job's operation can use = 5
[{'macineNumber': '2', 'processTime': '88'}, {'macineNumber': '3', 'processTime': '88'}, {'macineNumber': '5', 'processTime': '88'}, {'macineNumber': '9', 'processTime': '88'}, {'macineNumber': '10', 'processTime': '88'}]


<>:1: SyntaxWarning: invalid escape sequence '\H'
<>:1: SyntaxWarning: invalid escape sequence '\H'
C:\Users\okumo\AppData\Local\Temp\ipykernel_16816\644666744.py:1: SyntaxWarning: invalid escape sequence '\H'
  with open ("Dataset\Hurink_Data\\vdata\\abz5.fjs","r") as file:


## Encoding

In [22]:
# Initial population = [(J1, M3), (J1, M4), (J2, M1), (J3, M2), (J1, M1), ...]
# meaning J1 first operation use Machine 3 -> J2 first operation use Machine 4  -> J1 second operation......

# Because different JObs could have different
def initialize_population(NUM_JOBS):
    # Create an array with numbers 1 to 10 repeated 10 times
    base_array = np.tile(np.arange(1, ), NUM_JOBS) 
    population = []
    for _ in range(POP_SIZE):
        individual = np.random.permutation(base_array)
        population.append(individual)
    return population